In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import re
import streamlit as st

2023-06-16 10:26:13.742 WARNING matplotlib.font_manager: Matplotlib is building the font cache; this may take a moment.
2023-06-16 10:26:13.884 INFO    matplotlib.font_manager: generated new fontManager


In [ ]:
#Set the current directory for the kernel
path = os.getcwd()
files = os.listdir(path)
files_xls = [f for f in files if f[-3:] == 'XLS']

In [2]:
uploaded_files = st.file_uploader("Choose a file", accept_multiple_files=True)
files_xls = [f for f in uploaded_files if f[-3:] == 'XLS']

2023-06-16 10:35:05.285 
  command:

    streamlit run C:\Users\GadolS01\AppData\Local\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
#Set the file name pattern to sort the name through the code by date and time rather than sample number
pattern = r'\d{2}_\d{2}_\d{4}\s\d{2}_\d{2}_\d{2}'
filename_datetime_list = []
for filename in files_xls:
    match = re.search(pattern, filename)
    if match:
        datetime_str = match.group()
        datetime_obj = datetime.datetime.strptime(datetime_str, "%d_%m_%Y %H_%M_%S")
        filename_datetime_list.append((filename, datetime_obj))

# Sort the list of tuples based on the datetime object
sorted_filename_datetime_list = sorted(filename_datetime_list, key=lambda x: x[1])
sorted_filenames = [x[0] for x in sorted_filename_datetime_list]

In [ ]:
#This part of the code creates a full DataFrame which contains all the data from every excel file by keeping 
#the data sorted by worksheet
dfs = []
for filename in sorted_filenames:
    dfname = pd.ExcelFile(os.path.join(path, filename))
    for items in dfname.sheet_names:
        dfnew = pd.read_excel(dfname, sheet_name=items)
        dfnew['filename'] = filename  # add a new column to identify the source file
        dfs.append(dfnew)
data = pd.concat(dfs, ignore_index=True)

In [ ]:
#This part of the script rearranges the data according to the filename and keeps only the information about the 
#second occurrance of Nitrogen, which is the one needed
a = ['Hydrogen', 'Nitrogen', 'Carbon monoxide', 'Ethylene']
data = data.groupby('filename',as_index=False, sort=False).apply(lambda x: x[x['Name'].isin(a)].drop_duplicates(subset='Name', keep='last'))
data = data[['Name', 'Area', 'filename']].reset_index(drop=True)

In [ ]:
#UPDATE 1: Dealing with non-numerical data, such as N.D. which means the instrument does not detect the 
#target compound. If this piece of code is not run, the final table will be filled only with null values.
data["Area"] = data["Area"].replace(["N.D."], 0)
data["Area"] = pd.to_numeric(data["Area"])

In [ ]:
#Target table generation
pivot_table = data.pivot_table(index='filename', columns='Name', values='Area', sort=False)
column_order = ['Hydrogen', 'Nitrogen', 'Carbon monoxide', 'Ethylene']
pivot_table = pivot_table.reindex(columns=column_order)
pivot_table.to_excel("GC_outputs.xlsx")